## Setting up Environment

In [ ]:
### Dependencies
#
# pip install vizdoom + adding gitrepo to folder
# pip install gym

In [38]:
# Imports

from vizdoom import *
import random
import time
import numpy as np
from gym import Env
from gym.spaces import Discrete, Box
import cv2
from cv2 import resize

In [42]:
# Vizdoom OpenAI Gym Environment

class VizDoomGym(Env):

    def __init__(self, render=False):
        # Inherit from Env
        super().__init__()
        # Setup Game
        self.game = DoomGame()
        self.game.load_config("VizDoom/scenarios/basic.cfg")

        # check if render is active or not for faster compute
        if render == False:
            self.game.set_window_visibile(False)
        else:
            self.game.set_window_visible(True)

        self.game.init()

        # Create observation and action space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8)
        self.action_space = Discrete(3)
    
    # define a step in the environment
    def step(self, action):
        actions = np.identity(3)
        reward = self.game.make_action(actions[action],4)

        if self.game.get_state():
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = {"ammo":ammo}
        else:
            state = np.zeros(self.observation_space.shape)
            info = 0

        done = self.game.is_episode_finished()

        return state, reward, done, info

    # define how to render the game 
    def render():
        pass

    # define what happens at game start
    def reset(self):
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)

    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state

    # close game 
    def close(self):
        self.game.close()

In [44]:
env = VizDoomGym(render=True)
state= env.reset()

In [ ]:
env.step(2)

In [46]:
env.close()

## 2. Callback (Save Models)

In [ ]:
### Dependencies
#
# pytorch
# pip install stable-baselines3[extra]

In [14]:
import os
# PPO algorithmus
from stable_baselines3 import PPO
# Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

d:\Proggen\VirtualEnvironments\Anaconda\rl_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Callback to save our models
class TrainAndLoggingCallback(BaseCallback):
    # check_freq = after how many steps shall save model
    # save_path = path to save
    def __init__ (self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
    
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'VizDoom_Version_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [16]:
# Save Location
CHECKPOINT_DIR = './Training/Saved_Models'
LOG_DIR = './Training/Logs'

saveModel = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

## 3. Check Environment

In [17]:
from stable_baselines3.common import env_checker

In [45]:
env_checker.check_env(env)